In [0]:
%run "../Configuration"

In [0]:
# Define query to extract entityNames from sourceSystem table:
query = f"""
  SELECT entityNames
  FROM   sourceSystem
  WHERE  sourceSystemName = 'AdventureWorks'
  """

# Call query_entityNames function and load json:
entity_names = json.loads(query_entityNames(query))

In [0]:
# For each entity, create a dataFrame:
for entity in entity_names:
   if 'customer' in entity or 'product' in entity:
      # Read from SILVER path in metadata database:
      globals()[f"df_dim_{entity}"] = spark.read.format('delta').load(GOLD + entity_gold(entity))
      globals()[f'df_dim_{entity}'].createOrReplaceTempView(f"tv_dim_{entity}")
   else:
      globals()[f"df_fact_{entity}"] = spark.read.format('delta').load(GOLD + entity_gold(entity))
      globals()[f'df_fact_{entity}'].createOrReplaceTempView(f"tv_fact_{entity}")

In [0]:
%sql
SELECT * FROM tv_fact_sales_order_AW LIMIT 1;

In [0]:
%sql
SELECT * FROM tv_dim_customer_AW WHERE first_name = 'Rafael' AND last_name = 'Xu'

In [0]:
%sql
SELECT 
      COUNT(*) AS purchases
    , customer_id
FROM tv_fact_sales_order_AW
GROUP BY customer_id
ORDER BY COUNT(*) DESC;

In [0]:
dbutils.notebook.exit(0)